
Titanic Survival Prediction with SageMaker
This notebook walks through the steps to:
1. Read and split Titanic data into train/test sets.
2. Train an XGBoost model using SageMaker.
3. Make predictions on the test set and calculate accuracy.
4. Monitor the training and predictions with CloudWatch.


In [4]:
import boto3
import pandas as pd
from sklearn.model_selection import train_test_split
import sagemaker
from sagemaker import Session
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator
from sklearn.metrics import accuracy_score

In [7]:
bucket_name = 'test-bucket-hamady'
session = sagemaker.Session()
role = sagemaker.get_execution_role()

In [13]:

s3 = boto3.client('s3')
response = s3.get_object(Bucket=bucket_name, Key="processed/titanic.csv")
titanic_data = pd.read_csv(response['Body'])


X = titanic_data.drop("Survived", axis=1)
y = titanic_data["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


train_data = X_train.copy()
train_data['Survived'] = y_train
train_data = train_data[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked_Q", "Embarked_S"]]


train_data.to_csv("X_train_with_labels.csv", index=False)
X_test.to_csv("X_test.csv", index=False)
y_test.to_csv("y_test.csv", index=False)
s3.upload_file("X_train_with_labels.csv", bucket_name, "splitData/X_train_with_labels.csv")
s3.upload_file("X_test.csv", bucket_name, "splitData/X_test.csv")
s3.upload_file("y_test.csv", bucket_name, "splitData/y_test.csv")

In [15]:
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator
import sagemaker


container = sagemaker.image_uris.retrieve("xgboost", session.boto_region_name, version="1.5-1")
xgb = Estimator(
    image_uri=container,
    role=role,  
    instance_count=1,
    instance_type="ml.m5.xlarge",
    output_path=f"s3://{bucket_name}/xgboost-output",
    sagemaker_session=session
)


xgb.set_hyperparameters(objective="binary:logistic", num_round=100, max_depth=5, eta=0.2, subsample=0.8)


train_input = TrainingInput(
    f"s3://{bucket_name}/splitData/X_train_with_labels.csv",
    content_type="csv",
    input_mode="File",
  
)


xgb.fit({"train": train_input})


[01/04/25 05:14:02] INFO     Ignoring unnecessary instance type: None.                            ]8;id=253361;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=82217;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/image_uris.py#528\528]8;;\

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=407442;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=848531;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name:                                       ]8;id=309344;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=14991;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             sagemaker-xgboost-2025-01-04-05-14-02-316                                             

2025-01-04 05:14:02 Starting - Starting the training job...
2025-01-04 05:14:23 Starting - Preparing the instances for training...
2025-01-04 05:14:49 Downloading - Downloading input data...
2025-01-04 05:15:15 Downloading - Downloading the training image...
2025-01-04 05:16:00 Training - Training image download completed. Training in progress../miniconda3/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[2025-01-04 05:16:05.696 ip-10-0-187-160.eu-west-3.compute.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2025-01-04 05:16:05.720 ip-10-0-187-160.eu-west-3.compute.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2025-01-04:05:16:06:INFO] Imported framework sagemaker_xgboost_container.training
[2025-01-04:05:16:06:INFO] Failed to parse hyperparam

In [16]:
xgb_predictor = xgb.deploy(initial_instance_count=1, instance_type="ml.m5.large")


[01/04/25 05:18:57] INFO     Creating model with name: sagemaker-xgboost-2025-01-04-05-18-57-532    ]8;id=689283;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=248443;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[01/04/25 05:18:58] INFO     Creating endpoint-config with name                                     ]8;id=696482;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=395636;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\
                             sagemaker-xgboost-2025-01-04-05-18-57-532                                             

                    INFO     Creating endpoint with name sagemaker-xgboost-2025-01-04-05-18-57-532  ]8;id=762490;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=43334;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

------!